In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_pickle("../data/preprocess_train_dimbat_1.pkl")
df.sample(20)

,id,text,relevance,text_clean,lemmas,text_lemma
92200,646397350827659264,посвящаю <NUMBER> @comrinru@tommofuckhazza@ana...,0,посвящаю @comrinru@tommofuckhazza@anastasia ...,"[посвящаю, dash]",dash
92575,846820975069024256,"<USER> good evening, ma. couldn't get the fee ...",0,"good evening, ma. couldn't get the fee from a...","[good, evening, ma, fee, private, clinic, luth]",good evening ma fee private clinic luth
135022,74285979787202560,"<USER> oh, and i need to get some more money ...",0,"oh, and i need to get some more money  that...","[oh, I, need, money, good]",oh I need money good
97707,'348978138795098113',<HASHTAG> mhflood update - audio director of e...,1,mhflood update - audio director of emergency ...,"[mhflood, update, audio, director, emergency, ...",update audio director emergency management sou...
152965,458355813200183296,people 👫 who are so quick 💨💨 to walk away 🐾 <S...,0,people 👫 who are so quick 💨💨 to walk away 🐾 ...,"[people, quick, walk, away, intend, stay]",people quick walk away intend stay
52628,'324747652124266496',explosion at texas fertiliser plant: dozens of...,1,explosion at texas fertiliser plant: dozens of...,"[explosion, texas, fertiliser, plant, dozen, p...",explosion fertiliser plant people report injur...
99608,'348952324741402625',cleaning your flooded house? tips to stay safe...,1,cleaning your flooded house? tips to stay safe...,"[clean, flood, house, tip, stay, safe, healthy...",clean flood house tip stay safe healthy abfloo...
65469,704338227495305216,childhood crush since forever <URL>,0,childhood crush since forever,"[childhood, crush, forever]",childhood crush forever
12842,60350088769314816,just woke up now its time to do some history,0,just woke up now its time to do some history,"[wake, time, history]",wake time history
50525,567154070348382208,once i make it it's only gonna be my day <NUMBER>,0,once i make it it's only gonna be my day,"[I, day]",I day


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163718 entries, 0 to 163717
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   id          160958 non-null  object
 1   text        163718 non-null  object
 2   relevance   163718 non-null  int64 
 3   text_clean  163718 non-null  object
 4   lemmas      163718 non-null  object
 5   text_lemma  163718 non-null  object
dtypes: int64(1), object(5)
memory usage: 7.5+ MB


In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

/Users/xuxu/neuefische/NLPower-capstone-project/.venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from sklearn.model_selection import train_test_split
RSEED = 42

X_train, X_test, y_train, y_test = train_test_split(df['text_lemma'], df['relevance'], test_size=0.2, random_state=RSEED)

In [6]:
from datasets.dataset_dict import DatasetDict
from datasets import Dataset
dd = {'train':Dataset.from_dict({'text':X_train,'label':y_train}),
        'test':Dataset.from_dict({'text':X_test,'label':y_test})
       }
data = DatasetDict(dd)

In [7]:
data.shape

{'train': (130974, 2), 'test': (32744, 2)}

In [8]:
type(data)

datasets.dataset_dict.DatasetDict

In [9]:
data['train'][0]

{'text': 'release image suspect terrorist relation bombing drone htt',
 'label': 1}

In [10]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [11]:
tokenized_tweet = data.map(preprocess_function, batched=True)

100%|██████████| 33/33 [00:00<00:00, 77.45ba/s]


In [12]:
tokenized_tweet.shape

{'train': (130974, 4), 'test': (32744, 4)}

In [13]:
tokenized_tweet['train'][0]

{'text': 'release image suspect terrorist relation bombing drone htt',
 'label': 1,
 'input_ids': [101,
  2713,
  3746,
  8343,
  9452,
  7189,
  8647,
  18465,
  1044,
  4779,
  102],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [14]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [15]:
tf_train_set = tokenized_tweet["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids"],
    label_cols=["labels"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = tokenized_tweet["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids"],
    label_cols=["labels"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2022-08-01 12:39:08.341550: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-01 12:39:08.341926: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [16]:
tf_validation_set

<PrefetchDataset element_spec=({'input_ids': TensorSpec(shape=(None, None), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(None, None), dtype=tf.int64, name=None)}, TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [17]:
from transformers import create_optimizer
import tensorflow as tf

batch_size = 16
num_epochs = 5
batches_per_epoch = len(tokenized_tweet["train"]) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [18]:
from transformers import TFAutoModelForSequenceClassification

model_bert = TFAutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
model = TFAutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'vocab_layer_norm', 'vocab_transform', 'activation_13']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'dropout_19', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [19]:
import tensorflow as tf

model_bert.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                    metrics=tf.metrics.SparseCategoricalAccuracy())

In [20]:
history = model_bert.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=2)

Epoch 1/2


2022-08-01 12:39:53.220672: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-08-01 12:39:57.606404: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


8186/8186 [==============================] - ETA: 0s - loss: 0.1949 - sparse_categorical_accuracy: 0.9296

2022-08-01 22:09:14.605026: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


8186/8186 [==============================] - 34841s 4s/step - loss: 0.1949 - sparse_categorical_accuracy: 0.9296 - val_loss: 0.1786 - val_sparse_categorical_accuracy: 0.9362
Epoch 2/2
8186/8186 [==============================] - 103064s 13s/step - loss: 0.1400 - sparse_categorical_accuracy: 0.9516 - val_loss: 0.1995 - val_sparse_categorical_accuracy: 0.9324


In [22]:
model_bert.save_pretrained("../models/dimbat_disaster_distilbert_model")

In [23]:
tf_model = TFAutoModelForSequenceClassification.from_pretrained("../models/dimbat_disaster_distilbert_model")

Some layers from the model checkpoint at ../models/dimbat_disaster_distilbert_model were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at ../models/dimbat_disaster_distilbert_model and are newly initialized: ['dropout_59']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [55]:
input_text = "WHO alert over 10000000 monkeypox cases over the world"
input_text_tokenized = tokenizer.encode(input_text,
                                        truncation=True,
                                        return_tensors="tf")

In [56]:
pred_label = tf_model(input_text_tokenized)
pre_log = pred_label[0]
pre_probs = tf.nn.softmax(pre_log, axis=1).numpy()
print('The prediction probs are:')
print('not disater: ', pre_probs[0][0], '  disaster: ', pre_probs[0][1])

The prediction probs are:
not disater:  0.22179396   disaster:  0.77820605
